### Librairies

In [1]:
import os
import pandas as pd
import requests

### Authentification pour récupérer le token

In [2]:
headers_auth = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data_auth = {
    "grant_type": "client_credentials",
    "client_id": os.getenv("SPOTIFY_CLIENT_ID"),
    "client_secret": os.getenv("SPOTIFY_CLIENT_SECRET")
}

url_auth = "https://accounts.spotify.com/api/token"

try:
    response_auth = requests.post(url_auth, headers=headers_auth, data=data_auth)
    response_auth.raise_for_status()  # lève une exception si statut != 200

    # Extraction du token
    access_token = response_auth.json().get("access_token")

except requests.exceptions.RequestException as error:
    print("Exception during authentication:", error)
    access_token = None

### Utilisation du token pour appeler l’API artists

In [3]:
if access_token:
    print("Get artists informations ... ")

    headers_artists = {
        "Authorization": f"Bearer {access_token}"
    }

    artists_id = "7mRVAzlt1fAAR9Cut6Rq8c,7jy3rLJdDQY21OgRLCZ9sD,6olE6TJLqED3rqDCT0FyPh"  # Dave Grohl, Foo Fighters, Nirvana
    url_artists = f"https://api.spotify.com/v1/artists?ids={artists_id}"

    try:
        response_artists = requests.get(url_artists, headers=headers_artists)
        response_artists.raise_for_status()

        artists_json = response_artists.json()

        # On initialise une liste pour stocker les lignes du DataFrame
        artists_data = []

        for artist in artists_json["artists"]:

            data = {
                "id": artist["id"],
                "name": artist["name"],
                "url": artist["external_urls"]["spotify"],
                "image_url": artist["images"][0]["url"],
                "popularity": artist["popularity"],
                "genres": artist["genres"],  # laisse en liste
                "total_followers": artist["followers"]["total"],
                "uri": artist["uri"]
            }
            artists_data.append(data)

        # Création du DataFrame
        df_artists = pd.DataFrame(artists_data)

        # Sauvegarde CSV
        df_artists.to_csv("../datas/artists.csv", index=False)

        print("... OK")

    except requests.exceptions.RequestException as error:
        print("Exception during artists data retrieval:", error)

Get artists informations ... 
... OK
